# HAM10000数据库数据处理，分析与实验结果
leo, leoyhr@pku.edu.cn

In [182]:
import pandas as pd
import numpy as np

In [183]:
source_df = pd.read_csv('./HAM10000_metadata.csv')

In [39]:
source_df

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [184]:
source_df['class'] = pd.Categorical(source_df['dx']).codes

In [18]:
source_df['class'].value_counts()

5    6705
4    1113
2    1099
1     514
0     327
6     142
3     115
Name: class, dtype: int64

# 样本不均衡问题严重
考虑进行以下实验：
1. 不进行处理
2. 对每个类别最多采样300张图片



# 实验1
不进行特殊处理。为每一类都按比例选取70%作为训练数据，其余为测试数据

In [110]:
# 数据集划分
#为每一类都按比例选取70%作为训练数据，其余为测试数据

train_set_df = pd.DataFrame(columns=source_df.columns)
train_set = []
for i in range(7):
    train_set_df  = pd.concat([train_set_df, source_df[source_df['class'].isin([i])].sample(frac=0.7, random_state=1)])
    
test_set_df = source_df[~source_df['image_id'].isin(train_set_df['image_id'])]

# make lists
train_set = np.array(train_set_df[['image_id', 'class']]).tolist()
test_set = np.array(test_set_df[['image_id', 'class']]).tolist()

# save lists
train_set_df[['image_id', 'class']].to_csv('./train_set.csv',index=False)
test_set_df[['image_id', 'class']].to_csv('./test_set.csv', index=False)

In [289]:
len(train_set_df)

7010

In [290]:
len(test_set_df)

3005

In [93]:
3005+7010

10015

# 实验1结果：
Test for model ckp_step_30000, Test R1: 0.8878535628318787, R5: 0.9840266108512878

和kaggle的discussion上其他结果比较，已经是很好的结果。初步判断是因为使用了一个较大的模型以及较多的trick。

# 实验2
数据集划分：在训练数据中，每类封顶300张，测试数据不做限制。
#为每一类都按比例选取70%作为训练数据，其余为测试数据

In [291]:
# 数据集划分：在训练数据中，每类封顶300张，测试数据不做限制。
#为每一类都按比例选取70%作为训练数据，其余为测试数据

train_set_df = pd.read_csv('./train_set.csv')

# New constraint.
new_train_set_df = pd.DataFrame()
for i in range(7):
    new_train_set_df = pd.concat([new_train_set_df, train_set_df[train_set_df['class'].isin([i])][:300]])

# save new dataset
new_train_set_df[['image_id', 'class']].to_csv('./constrained_train_set.csv',index=False)

# 实验2结果：
Test for model ckp_step_30000, Test R1: 76.30615997314453, R5: 96.9051513671875.

量化指标不如原来的，考虑的原因是测试集中大类的数据也是主要组成。因此对算法的评价需要重新定义，究竟是要总体准确率高的，还是要每一类的预测准确率相近的。

# 实验3
不减少数据，而是增加小类的数据

In [285]:
train_set_df = pd.read_csv('./train_set.csv')

# New constraint.
new_train_set_df = pd.DataFrame()
for i in range(7):
    cat_set_df = train_set_df[train_set_df['class'].isin([i])]   
    new_train_set_df = pd.concat([new_train_set_df, train_set_df[train_set_df['class'].isin([i])].sample(n=4000, replace=True)])
    
# save new dataset
new_train_set_df[['image_id', 'class']].to_csv('./generated_train_set.csv',index=False)

In [288]:
new_train_set_df['class'].value_counts()

6    4000
5    4000
4    4000
3    4000
2    4000
1    4000
0    4000
Name: class, dtype: int64

# 实验3结果

分为两个子实验：
1. 使用identity loss
2. identity loss + triplet loss

结果：
1. Test for model ckp_step_30000, Test R1: 88.61897277832031, R5: 99.63394165039062
2. Test for model ckp_step_30000, Test R1: 88.35274505615234, R5: 99.80033111572266

分析：
1. 两个子实验来看，使用triplet loss并不能带来性能的显著提升。考虑是类别太少，考察类间关系的triplet loss意义不大。
2. 从实验1和实验3的结果对比来看，实验结果在Recall 1这个指标上并无大异，由此得到有两个关注点： **A . 测试集合的类别不均衡是否需要调整，如何设置合理的测试集； B. 是否需要在数据预处理等方面下更大的功夫，如尺寸金字塔等方法。 **

# 实验所用网络结构(对应所提供代码):
1. 基于Pytorch的Resnet50(last_stride = 1)， kaiming init, GAP w/ BN
2. CrossEntropy Loss 与 Triplet Loss, Adam优化器(w/ warmup)。详细参数见settings.py
3. 数据增强： 随机水平翻转+随机旋转（+-30 degrees）+随机裁剪， 网络输入尺寸为224*224